<a href="https://colab.research.google.com/github/rickiepark/llm-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 <br><<b><a href="<a href="http://tensorflow.blog/llm-from-scratch">밑바닥부터 만들면서 배우는 LLM</a></b>>의 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://tensorflow.blog/llm-from-scratch"><img src="https://tensorflowkorea.wordpress.com/wp-content/uploads/2025/09/ebb091ebb094eb8ba5llm_ebb3b8ecb185_ec959eeba9b4.jpg" width="100px"></a>
</td>
</tr>
</table>


# 2장: 텍스트 데이터 다루기

이 노트북에서 사용할 패키지:

In [23]:
!pip install tiktoken

In [55]:
from importlib.metadata import version

print("파이토치 버전:", version("torch"))
print("tiktoken 버전:", version("tiktoken"))

파이토치 버전: 2.8.0
tiktoken 버전: 0.12.0


- 이 장은 LLM의 입력 데이터를 준비하기 위한 데이터 전처리와 샘플링을 다룹니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="800px">

## 2.1 단어 임베딩 이해하기

- 많은 종류의 임베딩이 있지만 이 책에서는 텍스트 임베딩에 초점을 맞춥니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="700px">

- LLM은 (수천 개의 차원에 달하는) 고차원 공간의 임베딩을 다룹니다.
- 사람은 3차원 이하만 인식하고 고차원 공간을 시각화할 수 없기 때문에 아래 그림에서는 2차원 임베딩 공간을 보여줍니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="500px">

## 2.2 텍스트 토큰화하기

- 이 절에서 텍스트를 토큰화합니다. 즉 텍스트를 개별 단어와 구둣점 문자 같은 더 작은 단위로 나눕니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="500px">

- 사용할 텍스트 데이터는 이디스 워튼(Edith Wharton)의 단편 소설 "[The Verdict](https://en.wikisource.org/wiki/The_Verdict)"입니다.

In [25]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

___
**SSL 인증서 오류 문제 해결**

* 일부 독자들이 VSCode나 Jupyter에서 `urllib.request.urlretrieve`를 실행할 때 `ssl.SSLCertVerificationError: SSL: CERTIFICATE_VERIFY_FAILED` 오류를 겪는다고 합니다.
* 이는 보통 파이썬의 인증서 번들이 오래되었음을 의미합니다.

**해결 방법**

* 파이썬 3.9 이상을 사용하세요. 다음 코드를 실행하여 파이썬 버전을 확인할 수 있습니다:

```
import sys
print(sys.version)
```
* 인증서 번들을 업그레이드하세요:
  * `pip` 사용 시: `pip install --upgrade certifi`
  * `uv` 사용 시: `uv pip install --upgrade certifi`
* 업그레이드 후 Jupyter 커널을 재시작하세요.
* 위 코드를 실행한 후에도 ssl.SSLCertVerificationError가 계속 발생한다면, [원서 깃허브 PR](https://github.com/rasbt/LLMs-from-scratch/pull/403)에서 더 자세한 정보를 확인해 보세요.
___

In [26]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- LLM을 위해 이 텍스트를 토큰화하고 임베딩하는 것이 목표입니다.
- 샘플 텍스트를 사용해 간단한 토크나이저를 만들고 나중에 위 텍스트에 적용해 보겠습니다.
- 다음 정규 표현식은 공백을 기준으로 텍스트를 나눕니다.

In [27]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 공백으로만 나누지 않고 쉼표나 마침표도 나누고 싶습니다. 이를 위해 정규식을 수정해 보죠.

In [28]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 결과에 빈 문자열이 포함되어 있으므로 이를 삭제합니다.

In [29]:
# 각 항목에서 공백을 삭제하고 빈 문자열을 제외합니다.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 출력 결과가 좋아 보입니다. 추가적으로 물음표 같은 다른 구둣점 문자도 처리해 보죠.

In [30]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 좋네요. 이제 이 토큰화를 원시 텍스트에 적용해 보겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="500px">

In [31]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 총 토큰 개수를 확인해 보죠.

In [32]:
print(len(preprocessed))

4690


## 2.3 토큰을 토큰 ID로 변환하기

- 그다음 텍스트 토큰을 나주에 임베딩 층에서 처리할 수 있는 토큰 ID로 변환합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="700px">

- 고유한 토큰으로 구성된 어휘사전을 만듭니다.

In [33]:
set(preprocessed)

{'economy',
 'painted',
 'that',
 'knew',
 'other',
 'crowned',
 'women',
 'craft',
 'irony',
 'The',
 'purblind',
 'princely',
 'doing',
 'five',
 'made',
 'apparently',
 'without',
 'across',
 'Are',
 'discovery',
 'repeating',
 'big',
 'relatively',
 'At',
 'irrevocable',
 'frames',
 'ah',
 'slightly',
 'forehead',
 'clasping',
 'tried',
 'head',
 'mighty',
 'oak',
 'characteristic',
 'Poor',
 'mentioned',
 'nervous',
 'real',
 'desire',
 'rich',
 'dimmest',
 'forming',
 'shrug',
 'should',
 'tea',
 'going',
 'her',
 'life-likeness',
 'luncheon-table',
 'self-confident',
 'out',
 'set',
 'says',
 'hermit',
 'sunburnt',
 'least',
 'fact',
 'lightly',
 'get',
 'dingy',
 'extenuation',
 'stammer',
 'familiar',
 'raised',
 'purely',
 'boudoir',
 'happened',
 'now',
 'suspected',
 'absurdity',
 'swelling',
 'velveteen',
 'apparent',
 'loathing',
 ')',
 'bespoke',
 'It',
 'next',
 'same',
 'artist',
 'is',
 'circus-clown',
 'thither',
 'efforts',
 'somebody',
 'as',
 'dropped',
 'better',

In [34]:
sorted(set(preprocessed))

['!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '--',
 '.',
 ':',
 ';',
 '?',
 'A',
 'Ah',
 'Among',
 'And',
 'Are',
 'Arrt',
 'As',
 'At',
 'Be',
 'Begin',
 'Burlington',
 'But',
 'By',
 'Carlo',
 'Chicago',
 'Claude',
 'Come',
 'Croft',
 'Destroyed',
 'Devonshire',
 'Don',
 'Dubarry',
 'Emperors',
 'Florence',
 'For',
 'Gallery',
 'Gideon',
 'Gisburn',
 'Gisburns',
 'Grafton',
 'Greek',
 'Grindle',
 'Grindles',
 'HAD',
 'Had',
 'Hang',
 'Has',
 'He',
 'Her',
 'Hermia',
 'His',
 'How',
 'I',
 'If',
 'In',
 'It',
 'Jack',
 'Jove',
 'Just',
 'Lord',
 'Made',
 'Miss',
 'Money',
 'Monte',
 'Moon-dancers',
 'Mr',
 'Mrs',
 'My',
 'Never',
 'No',
 'Now',
 'Nutley',
 'Of',
 'Oh',
 'On',
 'Once',
 'Only',
 'Or',
 'Perhaps',
 'Poor',
 'Professional',
 'Renaissance',
 'Rickham',
 'Riviera',
 'Rome',
 'Russian',
 'Sevres',
 'She',
 'Stroud',
 'Strouds',
 'Suddenly',
 'That',
 'The',
 'Then',
 'There',
 'They',
 'This',
 'Those',
 'Though',
 'Thwing',
 'Thwings',
 'To',
 'Usually',
 'Venetian',
 'Victor',
 '

In [35]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [36]:
vocab = {token:integer for integer,token in enumerate(all_words)}
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'HAD': 44,
 'Had': 45,
 'Hang': 46,
 'Has': 47,
 'He': 48,
 'Her': 49,
 'Hermia': 50,
 'His': 51,
 'How': 52,
 'I': 53,
 'If': 54,
 'In': 55,
 'It': 56,
 'Jack': 57,
 'Jove': 58,
 'Just': 59,
 'Lord': 60,
 'Made': 61,
 'Miss': 62,
 'Money': 63,
 'Monte': 64,
 'Moon-dancers': 65,
 'Mr': 66,
 'Mrs': 67,
 'My': 68,
 'Never': 69,
 'No': 70,
 'Now': 71,
 'Nutley': 72,
 'Of': 73,
 'Oh': 74,
 'On': 75,
 'Once': 76,
 'Only': 77,
 '

- 어휘사전에 있는 처음 50개 항목을 확인합니다:

In [37]:
for i, item in enumerate(vocab.items()):  # item은 key, value 쌍입니다. i는 인덱스입니다.
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 아래 그림은 작은 어휘사전을 사용해 샘플 텍스트를 토큰화하는 과정을 보여줍니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="700px">

- 이를 모두 합쳐서 토큰화 클래스를 만듭니다.

In [38]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab  # vocab은 문자열을 정수로 매핑하는 딕셔너리입니다.
        self.int_to_str = {i:s for s,i in vocab.items()}  # 디코딩용 vocab 생성 

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # 'hello,. world'

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()  # 공백이 포함되어 있으면, 제거 후 저장 
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 함수는 텍스트를 토큰 ID로 바꿉니다.
- `decode` 함수는 토큰 ID를 텍스트로 바꿉니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="700px">

- 토크나이저를 사용해 텍스트를 정수로 인코딩(즉, 토큰화)합니다.
- 이 정수를 (나중에) 임베딩하여 LLM의 입력으로 사용할 수 있습니다.

In [39]:
tokenizer = SimpleTokenizerV1(vocab)  # vocab은 별도로 생성 한 딕셔너리입니다.

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 정수를 텍스트로 다시 디코딩할 수 있습니다.

In [40]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [41]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 특수 문맥 토큰 추가하기

- 알지 못하는 단어와 텍스트의 끝을 알리는 "특수" 토큰을 추가하면 도움이 됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="700px">

- 일부 토크나이저는 특수 토큰을 사용해 LLM이 추가적인 맥락을 이해하도록 돕습니다.
- 이런 특수 토큰에는 다음과 같은 것들이 있습니다.
  - `[BOS]` (beginning of sequence)는 텍스트의 시작을 표시합니다
  - `[EOS]` (end of sequence)는 텍스트의 끝을 표시합니다(서로 다른 두 개의 위키백과 문서나 책과 같이 관련이 없는 여러 텍스트를 연결할 때 사용합니다.)
  - `[PAD]` (padding) 하나 이상의 배치 크기로 LLM을 훈련할 때 배치 안에 길이가 다른 텍스트가 포함될 수 있습니다. 모든 텍스트의 길이를 동일하게 맞추기 위해 짧은 텍스트를 패딩 토큰을 사용해 배치에서 가장 긴 텍스트의 길이까지 확장합니다.
- `[UNK]`는 어휘 사전에 없는 단어를 나타냅니다.
- GPT-2는 위와 같은 토큰이 필요 없으며 복잡도를 줄이기 위해 `<|endoftext|>` 토큰만 사용합니다.
- `<|endoftext|>`는 위에서 언급한 `[EOS]` 토큰과 같습니다.
- GPT는 `<|endoftext|>`를 패딩에도 사용합니다(배치 입력에서 모델을 훈련할 때 마스크를 사용하기 때문에 패딩된 토큰에는 주의를 기울이지 않아 어떤 토큰을 사용하던지 관계가 없습니다).
- GPT-2는 어휘사전에 없는 토큰을 위해 `<UNK>` 토큰을 사용하지 않습니다. 대신 GPT-2는 바이트 페어 인코딩(BPE) 토크나이저를 사용해 단어를 부분단어로 분할합니다. 이에 대해서는 나중에 설명하겠습니다.

- 두 개의 독립된 텍스트 사이에 `<|endoftext|>` 토큰을 사용합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="800px">

- 다음과 같은 텍스트를 토큰화해 보죠:

In [42]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 단어 "Hello"가 어휘사전에 없기 때문에 오류가 발생합니다.
- 이를 처리하기 위해 알지 못하는 단어를 표현하는 `"<|unk|>"` 같은 특수 토큰을 어휘 사전에 추가할 수 있습니다.
- 어휘사전을 준비했으므로 GPT-2 훈련에서 텍스트의 끝을 나타내기 위해 사용된 `"<|endoftext|>"` 토큰을 추가하겠습니다(또한 훈련 데이터셋이 여러 개의 글이나 책으로 구성된 경우 연결된 텍스트 사이를 나타내기 위해서도 사용됩니다).

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}
vocab

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'HAD': 44,
 'Had': 45,
 'Hang': 46,
 'Has': 47,
 'He': 48,
 'Her': 49,
 'Hermia': 50,
 'His': 51,
 'How': 52,
 'I': 53,
 'If': 54,
 'In': 55,
 'It': 56,
 'Jack': 57,
 'Jove': 58,
 'Just': 59,
 'Lord': 60,
 'Made': 61,
 'Miss': 62,
 'Money': 63,
 'Monte': 64,
 'Moon-dancers': 65,
 'Mr': 66,
 'Mrs': 67,
 'My': 68,
 'Never': 69,
 'No': 70,
 'Now': 71,
 'Nutley': 72,
 'Of': 73,
 'Oh': 74,
 'On': 75,
 'Once': 76,
 'Only': 77,
 '

In [ ]:
len(vocab.items())

1132

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):  # vocab.items()는 key, value 쌍의 튜플 반환 
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 새로운 `<unk>` 토큰을 사용할 때와 방법을 알려 주기 위해 토크나이저를 수정합니다.

In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

수정된 토크나이저로 텍스트를 토큰화해 보죠:

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 바이트 페어 인코딩

- GPT-2는 바이트 페어 인코딩(BPE) 토크나이저를 사용합니다.
- 어휘사전에 없는 단어를 더 작은 부분단어나 개별 문자로 분할하여 처리할 수 있습니다.
- 예를 들어 GPT-2의 어휘사전에 단어 "unfamiliarword"가 없다면 이를 ["unfam", "iliar", "word"] 같이 토큰화할 수 있습니다. BPE의 훈련에 따라 결과가 달라질 수 있습니다.
- 원본 BPE 토크나이저는 다음 주소에 있습니다: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 이 장에서는 오픈AI의 오픈 소스 [tiktoken](https://github.com/openai/tiktoken) 라이브러리에서 제공하는 BPE 토크나이저를 사용합니다. 이 라이브러리는 계산 성능을 높이기 위해 핵심 알고리즘을 러스트(Rust)로 구현했습니다.
- [../02_bonus_bytepair-encoder](../02_bonus_bytepair-encoder) 폴더에 두 토크나이저 구현을 비교하는 노트북이 있습니다(tiktoken이 약 5배 빠릅니다).

In [58]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [59]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [60]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

In [61]:
print(tokenizer.encode(text, allowed_special='all'))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [62]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE 토크나이저는 알지 못하는 단어를 부분단어나 개별 문자로 분할합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="600px">

## 2.6 슬라이딩 윈도로 데이터 샘플링하기

- 한 번에 한 단어씩 생성하도록 LLM을 훈련하므로 시퀀스에 있는 다음 단어가 예측할 타깃이 되도록 훈련 데이터를 준비합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="600px">

In [63]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
print(enc_text[:10])

5145
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]


- 텍스트 청크에 대해 입력과 타깃이 있어야 합니다.
- 모델이 다음 단어를 예측해야 하므로 타깃은 오른쪽으로 한 토큰 이동한 입력입니다.

In [64]:
enc_sample = enc_text[50:]

In [65]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 한 번에 하나씩 예측은 다음과 같이 수행됩니다:

In [66]:
for i in range(1, context_size+1):
    context = enc_sample[:i]  # 이전 context에 포함된 전체 토큰
    desired = enc_sample[i]  # 예측하고자 하는 다음 토큰 

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [67]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 이후 장에서 어텐션 메커니즘을 소개한 후 다음 단어 예측을 다루겠습니다.
- 지금은 입력 데이터셋을 순회하면서 입력과 (한 토큰씩 뒤쳐진) 타깃을 반환하는 간단한 데이터 로더를 구현합니다.

- 슬라이딩 윈도를 사용해 한 토큰씩 위치를 이동합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="600px">

- 데이터셋과 입력 텍스트 데이터셋에서 청크를 추출하는 데이터 로더를 만듭니다.

In [68]:
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 전체 텍스트를 토큰화합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."

        # 슬라이딩 윈도를 사용해 책을 max_length 길이의 중첩된 시퀀스로 나눕니다.
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [69]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 토크나이저를 초기화합니다.
    tokenizer = tiktoken.get_encoding("gpt2")

    # 데이터셋을 만듭니다.
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 데이터 로더를 만듭니다.
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 문맥 크기를 4와 배치 크기 1로 데이터 로더를 테스트해 보죠.:

In [52]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [70]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 문맥 크기(이 경우 4)와 동일한 스트라이를 사용하는 예:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="600px">

- 배치 출력도 만들 수 있습니다.
- 배치 간에 중첩이 있으면 과대적합이 증가될 수 있으므로 스트라이드를 증가시킵니다.

In [72]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:\n", inputs)
print("\n타깃:\n", targets)

입력:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

타깃:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 토큰 임베딩 만들기

- LLM을 위한 데이터가 거의 준비되었습니다.
- 마지막으로 임베딩 층을 사용해 토큰을 연속적인 벡터 표현으로 임베딩해야 합니다.
- 일반적으로 이런 임베딩 층은 LLM의 일부이며 모델 훈련 과정에서 업데이트(훈련)됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="500px">

- (토큰화를 거친 후) 토큰 ID가 2, 3, 5, 1인 샘플을 가정해 보죠:

In [73]:
input_ids = torch.tensor([2, 3, 5, 1])

- 간단한 예를 위해 어휘사전에는 6개 단어만 들어 있고, 임베딩 크기는 3이라고 가정해 보겠습니다:

In [74]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 결과적으로 6x3 가중치 행렬이 만들어집니다:

In [75]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 원-핫 인코딩에 익숙한 사람을 위해서 설명하면 여기서 설명한 임베딩 층은 본질적으로 원-핫 인코딩 다음에 완전 연결 층을 두어 행렬 곱셈을 수행하는 것보다 효율적인 방법입니다. 깃허브 [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)에 있는 보너스 노트북에서 이를 확인할 수 있습니다.
- 임베딩 층은 원-핫 인코딩과 행렬 곱셈을 효율적으로 구현한 것에 지나지 않으므로 역전파로 최적화할 수 있는 신경망 층입니다.

- 토큰 ID 3을 3차원 벡터로 변환하는 방법은 다음과 같습니다:

In [76]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 출력 결과는 `embedding_layer` 가중치 행렬에 있는 네 번째 행입니다.
- `input_ids`에 있는 네 개의 값을 다음과 같인 모두 임베딩할 수 있습니다.

In [77]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 임베딩 층은 기본적으로 룩업 연산입니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="700px">

- 임베딩 층과 일반적인 선형 층을 비교하고 싶다면 다음 노트북을 참고하세요: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)

## 2.8 단어 위치 인코딩하기

- 임베딩 층은 토큰 ID를 입력 시퀀스에서 어떤 위치에 있던지 상관없이 동일한 벡터 표현으로 바꿉니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="600px">

- 위치 임베딩을 토큰 임베딩에 더해서 대규모 언어 모델을 위한 입력 임베딩을 만듭니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="600px">

- 바이트 페어 인코더의 어휘 사전 크기는 50,257입니다.
- 입력 토큰을 256차원의 벡터 표현으로 인코딩한다고 가정합니다.

In [78]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 데이터 로더에서 데이터를 샘플링한 다음 배치에 있는 각 샘플의 토큰을 256차원 벡터로 임베딩할 수 있습니다.
- 배치 크기가 8이고 샘플마다 4개의 토큰이 있다면 8 x 4 x 256 크기의 텐서가 만들어 집니다.

In [82]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [83]:
print("토큰 ID:\n", inputs)
print("\n입력 크기:\n", inputs.shape)

토큰 ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

입력 크기:
 torch.Size([8, 4])


In [84]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 임베딩 벡터의 값을 확인합니다.
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

- GPT-2는 절대 위치 임베딩을 사용하므로 또 다른 임베딩 층을 만들면 됩니다:

In [85]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 임베딩 층의 가중치를 확인합니다.
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[-0.6303, -0.4848, -0.1366,  ...,  1.0345, -0.5012,  1.1045],
        [ 0.2062,  0.6078,  0.7187,  ..., -0.4628, -0.2319,  1.1980],
        [ 0.5806, -1.3846,  0.3266,  ...,  0.8579,  0.5059,  1.0243],
        [ 1.4323,  0.2217,  0.8599,  ...,  0.4827,  0.8459,  1.3038]],
       requires_grad=True)


In [86]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# 위치 임베딩 값을 확인합니다.
print(pos_embeddings)

torch.Size([4, 256])
tensor([[-0.6303, -0.4848, -0.1366,  ...,  1.0345, -0.5012,  1.1045],
        [ 0.2062,  0.6078,  0.7187,  ..., -0.4628, -0.2319,  1.1980],
        [ 0.5806, -1.3846,  0.3266,  ...,  0.8579,  0.5059,  1.0243],
        [ 1.4323,  0.2217,  0.8599,  ...,  0.4827,  0.8459,  1.3038]],
       grad_fn=<EmbeddingBackward0>)


- LLM에 사용될 입력 임베딩을 만들기 위해 토큰 임베딩과 위치 임베딩을 더합니다:

In [87]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 입력 임베딩 값을 확인합니다.
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[-0.1390,  0.6390,  1.3222,  ...,  0.6350, -2.3747,  0.9599],
         [ 0.6543,  0.8614,  0.4532,  ...,  0.0369, -1.4310,  0.0136],
         [ 0.3299, -1.4393,  0.9953,  ...,  1.8197,  2.8795,  0.9715],
         [ 2.3781,  1.0874,  2.4790,  ...,  0.0283,  0.0999,  1.6521]],

        [[ 0.9158,  1.2520, -0.9214,  ...,  0.9341,  0.3572,  0.7624],
         [-1.6560,  0.4164,  0.3375,  ...,  0.6591, -0.5815,  1.8070],
         [ 2.5653, -2.0329,  0.1851,  ...,  0.4738, -0.4297,  2.4721],
         [ 2.3971,  1.5190, -0.7608,  ...,  1.6290,  2.4256,  1.7007]],

        [[-1.4016,  0.1724,  0.0297,  ...,  0.2302, -0.4470,  1.8471],
         [ 1.0107,  1.1125,  2.0109,  ...,  1.0020,  0.1778,  1.5185],
         [ 0.6601, -3.1482,  0.9016,  ...,  3.0402,  2.3289,  0.6608],
         [ 1.8590,  0.1569,  1.4285,  ..., -0.0382,  2.1524,  2.1511]],

        ...,

        [[-2.2458,  0.4762, -2.7803,  ...,  0.0701,  0.5877,  2.7428],
         [-0.1923, -0.3157, -0.59

- 입력 전처리 워크플로의 초기 단계에서 입력 텍스트를 별개의 토큰으로 분할합니다.
- 이 분할 단계 다음에 사전에 정의된 어휘사전을 기반으로 토큰을 토큰 ID로 변환합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="600px">

# 요약

이 장에서 구현한 데이터 로더의 간소화 버전이 [./dataloader.ipynb](./dataloader.ipynb) 노트북에 포함되어 있으며, 나중에 이어지는 장에서 GPT 모델을 훈련할 때 사용하겠습니다.

연습 문제 해답은 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 노트북을 참고하세요.

GPT-2의 토크나이저를 밑바닥부터 구현하고 훈련하는 방법이 궁금하다면 [바이트 페어 인코딩 토크나이저](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) 노트북을 참고하세요.